In [2]:
import requests
from bs4 import BeautifulSoup
import re

# URL for the Arabic Wikipedia article on Islam
url = 'https://ar.wikipedia.org/wiki/الإسلام'

# Headers to mimic a browser (helps avoid blocks)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def fetch_wikipedia_article(url, headers):
    """
    Fetch the Wikipedia article and extract clean text.
    """
    try:
        # Send GET request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise error for bad status codes
        
        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(response.content, 'lxml')
        
        # Find the main content div (Wikipedia's structure)
        content = soup.find('div', id='mw-content-text')
        if not content:
            raise ValueError("Could not find main content div")
        
        # Remove unwanted elements: infobox, references, navigation, etc.
        for element in content.find_all(['table', 'ul', 'ol', 'sup', 'div'], class_=re.compile(r'infobox|references|navbox|thumb|metadata')):
            element.decompose()
        
        # Extract text from paragraphs and headings
        text_elements = content.find_all(['p', 'h2', 'h3', 'h4'])
        full_text = []
        for elem in text_elements:
            # Get text, clean whitespace, and skip short/empty elements
            elem_text = elem.get_text(strip=True)
            if len(elem_text) > 50:  # Filter out very short snippets
                full_text.append(elem_text)
        
        # Join into full article text
        article_text = ''.join(full_text)
        
        # Basic cleaning: remove extra newlines and Arabic-specific artifacts if needed
        article_text = re.sub(r'\s*', '', article_text)
        
        return article_text
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page: {e}")
        return None
    except Exception as e:
        print(f"Parsing error: {e}")
        return None

# Fetch the article
article = fetch_wikipedia_article(url, headers)

if article:
    # Print a preview (first 1000 characters)
    print("Preview (first 1000 chars):")
    print(article[:1000] + "..." if len(article) > 1000 else article)
    
    # Print stats
    word_count = len(article.split())
    char_count = len(article)
    print(f"Full article stats:")
    print(f"- Words: ~{word_count:,} (Arabic token estimate)")
    print(f"- Characters: {char_count:,}")
    
    # Save to file for LLM use
    with open('islam_arabic_wikipedia.txt', 'w', encoding='utf-8') as f:
        f.write(article)
    print("Saved full text to 'islam_arabic_wikipedia.txt'")
    
    # For very long texts, you can chunk it for training
    paragraphs = article.split('')
    print(f"Number of paragraphs/sections: {len(paragraphs)}")
else:
    print("Failed to fetch article.")

Preview (first 1000 chars):
الإسلامدينسماويتوحيدي،يؤمنأتباعهبأنلاإلهإلاالله،[8]وأنمحمدًارسولالله.[9][10]ويعرفونباسمالمسلمين.[11]يُعلّمالإسلامأناللههورحيم،ولديهالقدرةالكلية،وهوواحد،[12]وقدأرشدالبشريةمنخلالالأنبياءوالرسل،والكتبالمقدسةوالآيات.[13]النصوصالأساسيةفيالإسلامهيالقرآن-الذيأوحيبهاللهإلىمحمدخلال23عامًافيمكةوالمدينة،[14][15]وينظرإليهالمسلمونعلىأنهكلمةاللهالحرفيةوالمعصومةعنالخطأ-والتعاليموالأمثلةالمعيارية(السنة)،والتيتشملالأحاديثالنبويةالخاصةبمحمد.يعتقدالمسلمونأنالإسلامهوالنسخةالكاملةالشاملةللعقيدةالتوحيديةالتيأُوحيَبهامراتعديدةإلىالأنبياء،كآدموإبراهيمونوحوداودويعقوبويوسفوصالحوهودويونسوموسىوزكرياويحييوعيسى.[16][17][18]يعتبرالمسلمونالقرآنالكريمالوحيالمطلقوالنهائيمنالله.[19]مثلالأديانالإبراهيميةالأخرى،فيالإسلامأيضاًحكمنهائييُمنحفيهالصالحونالجنةوغيرالصالحينالجحيم(جهنم).[20][21]تشملالمفاهيموالممارساتالدينيةأركانالإسلامالخمسة،وهيعباداتإجبارية،واتباعالشريعةالإسلامية،التيتمسكلجوانبالحياةوالمجتمعتقريبًا،منالأعمالالمصرفيةإلىالمرأةوالأخلاقوالبيئة.[22][23]مكةوالمدينةالمنورةوالقدسهيموطنلأقدسثلا

ValueError: empty separator